# First N-ODE example

A neural ODE is an ODE where a neural network defines its derivative function. $\dot{u} = NN(u)$

Docs: https://diffeqflux.sciml.ai/dev/examples/neural_ode/

In [ ]:
using Lux
using ComponentArrays
using DiffEqFlux
using OrdinaryDiffEq
using Optimization
using OptimizationOptimJL
using Random
using Plots

rng = Random.default_rng()

In [ ]:
u0 = Float32[2.0; 0.0]
datasize = 30
tspan = (0.0f0, 1.5f0)
tsteps = range(tspan[begin], tspan[end], length = datasize)

A sprial ODE to train against.

In [ ]:
const true_A = Float32[-0.1 2.0; -2.0 -0.1]

function trueODEfunc!(du, u, p, t)
    du .= ((u.^3)'true_A)'
end

In [ ]:
prob_trueode = ODEProblem(trueODEfunc!, u0, tspan)
ode_data = Array(solve(prob_trueode, Tsit5(), saveat = tsteps))

In [ ]:
nodeFunc = Lux.Chain(
    ActivationFunction(x -> x.^3),
    Lux.Dense(2, 50, tanh),
    Lux.Dense(50, 2)
)

p, st = Lux.setup(rng, nodeFunc)

In [ ]:
# Parameters for neural network
p

In [ ]:
# Use NeuroODE to construct an DE problem object
prob_node = NeuralODE(nodeFunc, tspan, Tsit5(), saveat = tsteps)

In [ ]:
function predict_neuralode(p)
    Array(prob_node(u0, p, st)[1])
end
  
function loss_neuralode(p)
    pred = predict_neuralode(p)
    loss = sum(abs2, ode_data .- pred)
    return loss, pred
end

# Callback function to observe training
anim = Animation()
callback = function (p, l, pred; doplot = true)
  # display(l)
  if doplot
	plt = scatter(tsteps, ode_data[1,:], label = "data")
    scatter!(plt, tsteps, pred[1,:], label = "prediction")
    frame(anim)
    # display(plot(plt))
  end
  return false
end

In [ ]:
# Train using the ADAM optimizer
adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x, p) -> loss_neuralode(x), adtype)
optprob = Optimization.OptimizationProblem(optf, ComponentArray(p))

In [ ]:
result_neuralode = Optimization.solve(
    optprob,
    ADAM(0.05),
    callback = callback,
    maxiters = 300
)

In [ ]:
# Retrain using the LBFGS optimizer
optprob2 = remake(optprob, u0 = result_neuralode.u)

result_neuralode2 = Optimization.solve(
    optprob2,
    LBFGS(),
    callback = callback,
    allow_f_increases = false
)

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()